In [1]:
from __future__ import (absolute_import, print_function, unicode_literals,
                        division)

import dropbot as db
import si_prefix as si

# Open DropBot connection

In [2]:
import base_node_rpc as bnr
import base_node_rpc.proxy

try:
    proxy.terminate()
except:
    pass
proxy = db.SerialProxy(ignore=[bnr.proxy.DeviceVersionMismatch])

proxy.enable_events()

proxy.update_state(hv_output_enabled=True, hv_output_selected=True,
                   voltage=100, frequency=10e3)

# Disable channels in contact with copper tape.
disabled_channels_mask_i = proxy.disabled_channels_mask
disabled_channels_mask_i[[89, 30]] = 1
proxy.disabled_channels_mask = disabled_channels_mask_i

# Benchmark Eigen library for computing $Y = (S_T S)^{-1} S_T M$

where:

 - $S$ is a $N \times N$ switching matrix encoding the actuation state of
   each channel during each measurement window in a measuring sequence,
   such that each row of $S$ corresponds to a window within a measurement
   period and each column corresponds to a _sensitive_ channel;
 - $Y$ is a $N \times 1$ matrix encoding the electrical admittance of
   each channel (where admittance is the inverse of the impedance) during
   each measurement period;
 - $M$ is a $N \times 1$ matrix M, containing a combined measurement for
   each row in $S$, corresponding to the _actuated_ channels in the row.
   Note that $SY = M$.

Benchmarking steps:

 1. Create a representative $N \times N$ switching matrix, $S$.
 2. Create a representative electrical admittance $N \times 1$ matrix, $Y$.
 3. Compute a mock capacitance measurement $N \times 1$ matrix, $M$,
    containing simulated a combined measurement for each row in $S$,
    corresponding to the actuated channels in the row.
 4. Compute transpose of $S$.
 5. **Compute the inverse of $S_T S$** in 3 loops, repeating ``repeats``
    times in each loop and recording the duration of each loop.
 6. Return the minimum duration of a single inverse computation.


## Maximum $S$ array size

Through empirical testing using `proxy._benchmark_eigen_inverse_float` and
`proxy._benchmark_eigen_inverse_double`:
 - **maximum size of $S$ is**:
   * **~50x50** when using `float` types for `Y` and `M`.
   * **~40x40** when using `double` types for `Y` and `M`.
 - **$(S_T S)^{-1}$ computation time** is **$\approx 5~µs$**

In [3]:
print('Inverse of $S_T S$ (float) took: {}s'
      .format(si.si_format(proxy._benchmark_eigen_inverse_float(40, 100))))
print('Inverse of $S_T S$ (double) took: {}s'
      .format(si.si_format(proxy._benchmark_eigen_inverse_double(40, 100))))

Inverse of $S_T S$ (float) took: 5.3 µs
Inverse of $S_T S$ (double) took: 5.3 µs
